# 手动实现贝叶斯算法


## 包引入

In [15]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.datasets as datasets
from sklearn.decomposition import PCA
import math

## 引入数据集

In [16]:
# 加载CIFAR-10数据集
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [24]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

## 超参数设定

In [28]:
# 最小风险矩阵
# m[i, j]代表把真实属于j类的样本归到i类中带来的损失
''' 0.3692
matrixLambda = np.array([
    [0, 1, 1, 2, 2, 2, 2, 2, 1, 1],
    [1, 0, 2, 2, 2, 2, 2, 2, 1, 1],
    [1, 2, 0, 1, 1, 1, 1, 1, 2, 2],
    [2, 2, 1, 0, 1, 1, 1, 1, 2, 2],
    [2, 2, 1, 1, 0, 1, 1, 1, 2, 2],
    [2, 2, 1, 1, 1, 0, 1, 1, 2, 2],
    [2, 2, 1, 1, 1, 1, 0, 1, 2, 2],
    [2, 2, 1, 1, 1, 1, 1, 0, 2, 2],
    [1, 1, 2, 2, 2, 2, 2, 2, 0, 1],
    [1, 1, 2, 2, 2, 2, 2, 2, 1, 0]
])

'''
# 0.3014
matrixLambda = np.array([
    [0, 2, 2, 1, 1, 1, 1, 1, 2, 2],
    [2, 0, 1, 1, 1, 1, 1, 1, 2, 2],
    [2, 1, 0, 2, 2, 2, 2, 2, 1, 1],
    [1, 1, 2, 0, 2, 2, 2, 2, 1, 1],
    [1, 1, 2, 2, 0, 2, 2, 2, 1, 1],
    [1, 1, 2, 2, 2, 0, 2, 2, 1, 1],
    [1, 1, 2, 2, 2, 2, 0, 2, 1, 1],
    [1, 1, 2, 2, 2, 2, 2, 0, 1, 1],
    [2, 2, 1, 1, 1, 1, 1, 1, 0, 2],
    [2, 2, 1, 1, 1, 1, 1, 1, 2, 0]
])

# # 0.3716
# matrixLambda = np.array([
#     [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#     [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
#     [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
#     [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
#     [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
#     [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
#     [1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
#     [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
# ])
# 数据特征降维
feature_count = 25

## 图片分类与类内计算

In [18]:
# 提取第c类图片
def extractClass(label, data, c):
    # 提取出第c类图片放到listC
    listC = []
    for i in range(len(label)):
        if label[i] == c:
            listC.append(list(data[i, : ]))
    return listC

# 计算类内均值 类内离散度 比例 新的训练数据 主成分分析模型
def calculateEC(x_train, y_train, feature_count):
    # matrixE[i][j]表示第i类样本在第j个属性上的均值
    matrixE = np.empty((10, feature_count))
    matrixC = np.empty((10, feature_count))

    pca = PCA(n_components = feature_count)
    new_train_data = pca.fit_transform(x_train)

    pc = [] # 先验概率
    for i in range(10):
        listC = extractClass(y_train, new_train_data, i)
        pc.append(len(listC) / len(y_train))
        for j in range(feature_count):
            list1 = []
            for k in range(len(listC)):
                list1.append(listC[k][j])
            matrixE[i, j] = np.mean(list1)
            matrixC[i, j] = np.cov(list1)
    return matrixE, matrixC, pc, new_train_data, pca

In [19]:
# 根据最小风险贝叶斯模型进行分类
def calculatePostrior(matrixE, matrixC, pc, test_data, feature_count):
    risk = []
    for i in range(10):
        r = 0
        for j in range(10):
            p_x_wj = 1
            for k in range(feature_count):
                p_x_wj = p_x_wj * math.exp(- (test_data[k] - matrixE[j][k]) ** 2 / (2 * matrixC[j][k])) / math.sqrt(2 * math.pi * matrixC[j][k])
            r += matrixLambda[i][j] * pc[i] * p_x_wj
        risk.append(r)
    return risk.index(min(risk))

In [29]:
if __name__ == '__main__':
    matrixE, matrixC, pc, new_train_data, pca = calculateEC(x_train, y_train, feature_count)
    new_test_data = pca.transform(x_test)

    # 进行类别预测
    pre_label = []
    for i in range(len(y_test)):
        pre_label.append(calculatePostrior(matrixE, matrixC, pc, new_test_data[i, : ], feature_count))

    error_count = 0
    for i in range(len(pre_label)):
        if pre_label[i] != y_test[i]:
            error_count += 1

    print('Total test number is %d' % len(y_test))
    print('Test accuracy is %f' % (1 - error_count / len(y_test)))

Total test number is 10000
Test accuracy is 0.301900
